In [5]:
import pandas as pd
import numpy as np

import xgboost as xgb
import os

from tqdm.notebook import tqdm

In [6]:
from google.colab import drive
drive.mount('/content/drive')

base_path = '/content/drive/My Drive/ir/final'
os.chdir(base_path)

Mounted at /content/drive


In [2]:
!pip3 uninstall numpy
!pip3 install numpy==1.20

Found existing installation: numpy 1.21.5
Uninstalling numpy-1.21.5:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/f2py3
    /usr/local/bin/f2py3.7
    /usr/local/lib/python3.7/dist-packages/numpy-1.21.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libgfortran-2e0d59d6.so.5.0.0
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libopenblasp-r0-2d23e62b.3.17.so
    /usr/local/lib/python3.7/dist-packages/numpy.libs/libquadmath-2d0c479f.so.0.0.0
    /usr/local/lib/python3.7/dist-packages/numpy/*
Proceed (y/n)? y
  Successfully uninstalled numpy-1.21.5
     |████████████████████████████████| 15.3 MB 14.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.20.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 whic

In [3]:
!pip3 uninstall folium
!pip3 install folium==0.2.1

Found existing installation: folium 0.8.3
Uninstalling folium-0.8.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/folium-0.8.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/folium/*
Proceed (y/n)? y
  Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 69 kB 6.2 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=54c641a7a4f38d755c9d1e1e967018bf8b758a9eb24facdbfdfd126355f32f62
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium


In [5]:
!pip3 uninstall imgaug
!pip3 install imgaug==0.2.5

Found existing installation: imgaug 0.2.7
Uninstalling imgaug-0.2.7:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/imgaug-0.2.7.dist-info/*
    /usr/local/lib/python3.7/dist-packages/imgaug/*
Proceed (y/n)? y
  Successfully uninstalled imgaug-0.2.7
     |████████████████████████████████| 562 kB 13.8 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.5-py3-none-any.whl size=561438 sha256=e9bb811aa432cd5b5900e24ebdc773399c11a5b154baa61ac16d4e389e352224
  Stored in directory: /root/.cache/pip/wheels/60/dd/38/d1dc2cad2b6a66dc0249261004990bccb0f27985c74ba26e49
Successfully built imgaug


In [37]:
!pip3 install tensorflow_text>=2.0.0rc0

In [36]:
import tensorflow as tf

In [1]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

In [2]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")
embed_qa = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3")

In [ ]:
from helpers_sem import cut_query

In [12]:
query = ['в какую государственную структуру входит служба по экономическим расследованиям казахстан']
titles = ['как называется служба по экономическим расследования', 'в каких государственных структурах лучше работать в казахстане']

In [13]:
query_result = embed(query)
titles_result = embed(titles)
sim_matrix = np.inner(query_result, titles_result)
sim_matrix

array([[0.59951204, 0.59767425]], dtype=float32)

### Вспомогательные функции

In [3]:
def get_sim_qa(data_df, f_input_name, f_query_name, f_context_name=None, need_cut=False):
  data_df[f_input_name].fillna('', inplace=True)
  if f_context_name:
    data_df[f_context_name].fillna('', inplace=True)
  sim_use_qa_list = []
  qids = np.unique(data_df['query_id'], return_counts=False)
  for qid in tqdm(qids, total=len(qids)):
    query = queries_df[queries_df.id == qid][f_query_name].values[0]
    inputs = data_df[data_df.query_id == qid][f_input_name].values
    if f_context_name:
      contexts = data_df[data_df.query_id == qid][f_context_name].values
    else:
      contexts = inputs
    if need_cut:
      inputs = list(map(lambda x: cut_query(x), inputs))
    query_result = embed_qa.signatures['question_encoder'](tf.constant(query))
    inputs_result = embed_qa.signatures['response_encoder'](input=tf.constant(inputs), context=tf.constant(contexts))
    sim_matrix = np.inner(query_result['outputs'], inputs_result['outputs'])
    sim_use_qa_list.extend(list(sim_matrix[0]))
  return sim_use_qa_list
  
def get_sim(data_df, f_input_name, f_query_name, need_cut=False):
  data_df[f_input_name].fillna('', inplace=True)
  sim_use_list = []
  qids = np.unique(data_df['query_id'], return_counts=False)
  for qid in tqdm(qids, total=len(qids)):
    query = queries_df[queries_df.id == qid][f_query_name].values[0]
    inputs = data_df[data_df.query_id == qid][f_input_name].values
    if need_cut:
      inputs = list(map(lambda x: cut_query(x), inputs))
    query_result = embed(query)
    inputs_result = embed(inputs)
    sim_matrix = np.inner(query_result, inputs_result)
    sim_use_list.extend(list(sim_matrix[0]))
  return sim_use_list

### Загрузка данных

In [7]:
queries_df = pd.read_csv('corrected_queries.tsv', sep='\t')

### Расчеты для запросов

In [8]:
click_data_df = pd.read_csv('click_data_df.csv', sep='\t')
click_data_df.sort_values(by=['query_id', 'c_query_id'], inplace=True)

In [9]:
click_data_df['sim_q'] = get_sim(click_data_df, 'c_query', 'corrected_query')

  0%|          | 0/6311 [00:00<?, ?it/s]

In [32]:
click_data_df[click_data_df.query_id == 14]

,query_id,c_query_id,c_query,filename,log_id_start,c_query_weight,sim_q
140,14,4365549,по достижении какого возраста можно без опекун...,part-m-00056.bz2,10164069,191549.11,0.277628
141,14,22768907,при ипотеке в сбербланке можно ли выбирать стр...,part-m-00177.bz2,49357396,212685.83,0.285210
142,14,38842644,можно ли изменить фамилию мужчине по своему ус...,part-m-01010.bz2,310087109,232685.83,0.536504
143,14,38844330,можно ли изменять виды работ по 44 фз при закл...,part-m-01010.bz2,310087747,189113.77,0.579486
144,14,38844419,можно ли изменять состав работ по 44 фз,part-m-01010.bz2,310087863,188615.75,0.605263
145,14,38844427,можно ли изменять существенные условия по 223 фз,part-m-01010.bz2,310087873,188615.75,0.552105
146,14,38844498,можно ли изменять типовой контракт по 44 фз,part-m-01010.bz2,310087892,188615.75,0.577861
147,14,38844874,можно ли изменять цену договора по аукциону по...,part-m-01010.bz2,310087925,189113.77,0.529294
148,14,64056286,31 можно ли изменить оформление списков по сво...,part-m-01619.bz2,500221046,371597.28,0.956654
149,14,64056288,31 можно ли изменять оформление списков по сво...,part-m-01619.bz2,500221049,415570.25,1.000000


In [34]:
len(np.unique(click_data_df[click_data_df.sim_q >= 0.3].query_id, return_counts=False))

6282

In [35]:
click_data_df.to_csv('click_data_df.csv', index=False, sep='\t')

### Расчеты для train

In [ ]:
train_use_df = pd.read_csv('train_use_df.csv', sep='\t')
train_use_df.sort_values(by=['query_id', 'doc_id'], inplace=True)

In [ ]:
train_use_df['sim_use_qt'] = get_sim(train_use_df, 'doc_title_norm', 'query_norm')
train_use_df['sim_use_qt_un'] = get_sim(train_use_df, 'doc_title', 'corrected_query')
train_use_df['sim_use_qa_qtt_un'] = get_sim_qa(train_use_df, 'doc_title', 'corrected_query')
train_use_df['sim_use_qs_512_un'] = get_sim(train_use_df, 'doc_snippet_512', 'corrected_query')
train_use_df['sim_use_qs_1024_un'] = get_sim(train_use_df, 'doc_snippet_1024', 'corrected_query')
train_use_df['sim_use_qa_qts_512_un'] = get_sim_qa(train_use_df, 'doc_title', 'corrected_query', f_context_name='doc_snippet_512')
train_use_df['sim_use_qa_qts_1024_un'] = get_sim_qa(train_use_df, 'doc_title', 'corrected_query', f_context_name='doc_snippet_1024')
train_use_df['sim_use_qa_qss_512_un'] = get_sim_qa(train_use_df, 'doc_snippet_512', 'corrected_query')
train_use_df['sim_use_qa_qss_1024_un'] = get_sim_qa(train_use_df, 'doc_snippet_1024', 'corrected_query')

  0%|          | 0/2111 [00:00<?, ?it/s]

In [ ]:
train_use_df.to_csv('train_use_df.csv', index=False, sep='\t')

### Расчет для test

In [ ]:
test_use_df = pd.read_csv('test_use_df.csv', sep='\t')
test_use_df.sort_values(by=['query_id', 'doc_id'], inplace=True)

In [ ]:
test_use_df['sim_use_qt'] = get_sim(test_use_df, 'doc_title_norm', 'query_norm')
test_use_df['sim_use_qt_un'] = get_sim(test_use_df, 'doc_title', 'corrected_query', need_cut=True)
test_use_df['sim_use_qa_qtt_un'] = get_sim_qa(test_use_df, 'doc_title', 'corrected_query', need_cut=True)
test_use_df['sim_use_qs_512_un'] = get_sim(test_use_df, 'doc_snippet_512', 'corrected_query')
test_use_df['sim_use_qs_1024_un'] = get_sim(test_use_df, 'doc_snippet_1024', 'corrected_query')
test_use_df['sim_use_qa_qts_512_un'] = get_sim_qa(test_use_df, 'doc_title', 'corrected_query', f_context_name='doc_snippet_512', need_cut=True)
test_use_df['sim_use_qa_qts_1024_un'] = get_sim_qa(test_use_df, 'doc_title', 'corrected_query', f_context_name='doc_snippet_1024', need_cut=True)
test_use_df['sim_use_qa_qss_512_un'] = get_sim_qa(test_use_df, 'doc_snippet_512', 'corrected_query')
test_use_df['sim_use_qa_qss_1024_un'] = get_sim_qa(test_use_df, 'doc_snippet_1024', 'corrected_query')

In [ ]:
test_use_df.to_csv('test_use_df.csv', index=False, sep='\t')